In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd

from config import ACCESS_KEY,SECRET_KEY,TOKEN,APP_METRICA_OAUTH
local_folder = '/home/boris/Documents/Work/mobile rep cifrall'

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    # now = datetime.datetime.strptime('2025-01-01 00:00:00','%Y-%m-%d %H:%M:%S')
    now = datetime.datetime.now(pytz.timezone(time_zone))
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)

    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d')
            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']

limit = 100
offset = 1

headers={'Authorization':APP_METRICA_OAUTH,'Host':'api.appmetrica.yandex.ru'}
limit = 100
offset = 1


def get_request_metrica(api_html_metrica):
    response = requests.get(
        api_html_metrica,
        headers = headers
    )
    return response

token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"


def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

def function_lunch(FOLDER,parameters):
    global offset, query_name, query_description, query_text, request_id
    offset = 0
    query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
    query_description = 'тестовый запрос'#описание, которое появляется в запросах
    query_text = open(f'{FOLDER}.txt','r').read().format(*parameters)

    request_id = create_query()
    while str(get_request(0)) == '<Response [400]>':     
        time.sleep(10)

    write_temp_file()
    temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
    create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
    temp_file_pd.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
    remove_temp_files()


def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'{local_folder}/Ready report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"mobile_report_comerce_2/year={str(year)}/{str(month)}.csv"
    )

In [2]:
FOLDER = 'total_activated_account_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [3]:
FOLDER = 'total_active_users_rep_mobile_cifral'
parameters = (now_date,start_of_month_data,yesterday_data)
function_lunch(FOLDER, parameters)

In [4]:
FOLDER = 'active_users_in_flats_rep_mobile_cifral'
parameters = (now_date,start_of_month_data,yesterday_data)
function_lunch(FOLDER, parameters)

In [5]:
FOLDER = 'total_account_count_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [6]:
FOLDER = 'DAF_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [7]:
FOLDER = 'MAF_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [8]:
FOLDER = 'app_in_flats_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'flats_in_sputnic_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'all_flats_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'paying_active_flats_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'mobile_report_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data,month,year)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'mobile_report_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data,month,year)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'service_payments_rep_mobule_cifral'
service_payments = pd.DataFrame({
    'service_payments': 0
            },index=[0])

create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
service_payments.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")

In [ ]:
FOLDER = 'subscriptions_report_comerce_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data,year,month)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'total_active_users_per_day_rep_mobile_cifral'
parameters = (now_date,start_of_month_data,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
FOLDER = 'Units_on_SK platform_(monetization)_rep_mobile_cifral'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [ ]:
# def get_now_datetime_str(): # получаем актуальное время
#     time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
#     #now = datetime.datetime.strptime('2023-07-01 00:00:00','%Y-%m-%d %H:%M:%S')
#     now = datetime.datetime.now(pytz.timezone(time_zone))    
#     yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
#     days_to_subtract = yesterday.day - 1
#     start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
#     yesterday = now - datetime.timedelta(days=1)

#     return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
#             'now_date':now.strftime('%Y-%m-%d'),
#             'yesterday_data':yesterday.strftime('%Y-%m-%d'),
#             'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
#             'year':yesterday.strftime('%Y'),
#             'month':yesterday.strftime('%m'),
#             'day':yesterday.strftime('%d'),
#             'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d')

#             }

# # Переменные, которые могут быть использованы в разоичных запросах. 
# now = get_now_datetime_str()['now']
# # Даты ниже отражают вчерашний день
# yesterday = get_now_datetime_str()['yesterday']
# yesterday_data = get_now_datetime_str()['yesterday_data']
# day = get_now_datetime_str()['day'] # 
# month = get_now_datetime_str()['month']
# year = get_now_datetime_str()['year']
# start_of_month_data = get_now_datetime_str()['start_of_month_data']
# now_date = get_now_datetime_str()['now_date']

# limit = 100
# offset = 1

# print(now)
# print(now_date)
# print(yesterday)
# print(yesterday_data)
# print(start_of_month_data)

# print(day)
# print(month)
# print(year)


# Concat

In [ ]:
service_payments = pd.read_csv(f'{local_folder}/service_payments_rep_mobule_cifral/{int(year)}/{int(month)}.csv', sep=',')
service_payments = service_payments['service_payments'].to_frame(name = 'service_payments')
service_payments

,service_payments
0,0


In [ ]:
mobile_report = pd.read_csv(f'{local_folder}/mobile_report_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
mobile_report

,Первая дата,Месяц,Год,IOS_PL,appstore_count_69,appstore_count_69_refunded,appstore_count_499,appstore_count_499_refunded,appstore_count_2390,appstore_count_2390_refunded,...,yookassa_count_35_refunded,yookassa_count_1,yookassa_count_1_refunded,yookassa_count_499,yookassa_count_499_refunded,yookassa_count_249,yookassa_count_249_refunded,yookassa_count_2390,yookassa_count_2390_refunded,refunded_amount_yookassa
0,'2024-12-01',12,2024,'',366,0,0,0,0,0,...,0,581,26,0,0,0,0,0,0,209


In [ ]:
subscriptions_report_comerce = pd.read_csv(f'{local_folder}/subscriptions_report_comerce_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
#subscriptions_report_comerce = subscriptions_report_comerce[['failed_subscriptions_renewed','canceled_subscriptions_renewed','paying_users','paying_users_standart','paying_users_standart_yakassa','paying_users_standart_appstore','paying_users_standart_ios_from_cart','new_subscriptions','failed_subscriptions','canceled_subscriptions','renewed_subscriptions']]
subscriptions_report_comerce

,paying_users,paying_users_standart,paying_users_standart_yakassa1,paying_users_standart_yakassa,paying_users_standart_appstore,paying_users_standart_ios_from_cart,renewed_subscriptions,new_subscriptions,new_subscriptions_activated,failed_subscriptions,failed_subscriptions_renewed,canceled_subscriptions,canceled_subscriptions_renewed
0,5404,5404,0,3404,325,1433,4818,634,586,392,64,50,66


In [ ]:
units_on_SK_platform = pd.read_csv(f'{local_folder}/Units_on_SK platform_(monetization)_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
units_on_SK_platform

,Units on SK platform,Units stricted monetization,Units free monetization
0,6458,4691,1767


In [ ]:
total_account_count = pd.read_csv(f'{local_folder}/total_account_count_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
total_account_count

,total_account_count
0,201685


In [ ]:
total_activated_account = pd.read_csv(f'{local_folder}/total_activated_account_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
total_activated_account

,total_activated_account,total_activated_account_monetization
0,200158,93043


In [ ]:
total_active_users = pd.read_csv(f'{local_folder}/total_active_users_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
total_active_users

,total_active_users,total_active_users_monetization
0,114324,50626


In [ ]:
all_flats = pd.read_csv(f'{local_folder}/all_flats_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
all_flats

,all_flats,freemonetization_flats,stricted_flats,subsc_restrict_is_null_flats
0,126722,55409,71313,0


In [ ]:
flats_in_sputnic = pd.read_csv(f'{local_folder}/flats_in_sputnic_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
flats_in_sputnic

,flats_in_sputnic
0,98556


In [ ]:
maf = pd.read_csv(f'{local_folder}/MAF_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
maf

,MAF,stricted_MAF,freemonetization_MAF
0,70932,39647,31285


In [ ]:
daf = pd.read_csv(f'{local_folder}/DAF_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',').round().astype('int')
daf

,DAF,stricted_DAF,freemonetization_DAF
0,29689,16849,12840


In [ ]:
paying_active_flats = pd.read_csv(f'{local_folder}/paying_active_flats_rep_mobile_cifral/{int(year)}/{int(month)}.csv', sep=',')
paying_active_flats

,paying_active_flats
0,5274


In [ ]:
concated = pd.concat([
                      service_payments['service_payments'],
                      mobile_report,
                      subscriptions_report_comerce, 
                      units_on_SK_platform,
                      total_account_count,
                      total_activated_account,
                      total_active_users,
                      all_flats,
                      flats_in_sputnic,
                      maf,
                      daf,
                      paying_active_flats,
                      ]
                     , axis=1)
concated

,service_payments,Первая дата,Месяц,Год,IOS_PL,appstore_count_69,appstore_count_69_refunded,appstore_count_499,appstore_count_499_refunded,appstore_count_2390,...,stricted_flats,subsc_restrict_is_null_flats,flats_in_sputnic,MAF,stricted_MAF,freemonetization_MAF,DAF,stricted_DAF,freemonetization_DAF,paying_active_flats
0,0,'2024-12-01',12,2024,'',366,0,0,0,0,...,71313,0,98556,70932,39647,31285,29689,16849,12840,5274


In [ ]:
concated.to_csv(f'{local_folder}/Ready report/{int(year)}/{int(month)}.csv', sep=',', index=False)

OSError: Cannot save file into a non-existent directory: '/home/boris/Documents/Work/mobile rep cifrall/Ready report/2024'

In [ ]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'{local_folder}/Ready report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"mobile_report_comerce_cifral/year={str(year)}/{str(month)}.csv"
    )

In [ ]:
get_s3_instance()
upload_dump_to_s3()
